# Lab 11: Misc Functions

In [18]:
library(tidyverse)
library(nycflights13)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.5
✔ tidyr   0.8.1     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘tibble’ was built under R version 3.4.3”Warning message:
“package ‘tidyr’ was built under R version 3.4.4”Warning message:
“package ‘purrr’ was built under R version 3.4.4”Warning message:
“package ‘dplyr’ was built under R version 3.4.4”Warning message:
“package ‘forcats’ was built under R version 3.4.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘nycflights13’ was built under R version 3.4.4”

## Review

Remember that **lists** are sequences where the elements are allowed to be different data types (including vectors or even other lists). You will usually want to name your list elements.

In [210]:
x = list(a='a', b=FALSE, c=1:3, d=list(first=c(1, 3, 5), second=c('a', 'b', 'c')), e=pi)
print(names(x))
print(x)

[1] "a" "b" "c" "d" "e"
$a
[1] "a"

$b
[1] FALSE

$c
[1] 1 2 3

$d
$d$first
[1] 1 3 5

$d$second
[1] "a" "b" "c"


$e
[1] 3.141593



There are three ways to subset or extract elements from a list:
* `[...]` will extract a sublist. Note that the result of this will always be another list. Integer, logical, or character vectors can be used.
* `[[...]]` will extract a single element. Either the index or name of the desired element can be provided.
* `$a` will also extract a single element. Note that this requires a named list, and the name must be used. 

### Functional Programming

Tidyverse contains a suite of functions used for functional programming in `purrr` ([documentation](https://purrr.tidyverse.org/index.html)).

Functional programming is generally built on three main operations:
* `map`
* `keep` (usually known as `filter` in other languages)
* `reduce`

Note that `map` always returns a list, if you want a vector, then use the functions `map_lgl`, `map_int`, `map_dbl`, or `map_chr` for logicals, integers, doubles/floats, and strings, respectively.

In [183]:
map(1:5, function(x) x^2) %>% print

[[1]]
[1] 1

[[2]]
[1] 4

[[3]]
[1] 9

[[4]]
[1] 16

[[5]]
[1] 25



In [188]:
map_dbl(1:5, function(x) x^2)

[1]  1  4  9 16 25

In [193]:
keep(1:5, function(x) x %% 2 == 0)

[1] 2 4

In [186]:
reduce(1:5, function(x, y) x + y)

[1] 15

In [187]:
accumulate(1:5, function(x, y) x + y)

[1]  1  3  6 10 15

In [196]:
map_dbl(1:5, ~ .^2)
keep(1:5, ~ . %% 2 == 0)
reduce(1:5, ~ .x + .y)

[1]  1  4  9 16 25

[1] 2 4

[1] 15

Remember that dataframes are also lists where each element is a vector (i.e. a column of the data), so when applied to dataframes these will apply to each column of the data.

In [198]:
mtcars %>% map_dbl(sum) %>% keep(~ . > 200) %>% print

    mpg    disp      hp    qsec 
 642.90 7383.10 4694.00  571.16 


## Q1: Working with logicals

### 1.1

Given a logical vector, consider a function that returns `TRUE` if every element is `TRUE`, and returns `FALSE` if any element is `FALSE`. Have this function ignore missing values (don't worry about empty vectors or vectors will all missing values).

Write the function `all_iter` which uses iteration/loops to perform this calculation.

Write the function `all_func` which uses functional programming tools.

In [30]:
all_iter = function(x) {
    for (val in x) {
        if (!is.na(val)) {
            if (!val) {
                return(FALSE)
            }
        }
    }
    return(TRUE)
}

In [31]:
all_func = function(x) {
    x %>% keep(~!is.na(.)) %>% reduce(`&`)
}

In [32]:
stopifnot(all_iter(c(TRUE, TRUE, TRUE)))
stopifnot(!all_iter(c(TRUE, TRUE, FALSE)))
stopifnot(!all_iter(c(FALSE, FALSE, FALSE)))
stopifnot(all_iter(c(TRUE, TRUE, NA)))
stopifnot(!all_iter(c(TRUE, FALSE, NA)))
stopifnot(all_iter(c(NA, TRUE, NA)))

stopifnot(all_func(c(TRUE, TRUE, TRUE)))
stopifnot(!all_func(c(TRUE, TRUE, FALSE)))
stopifnot(!all_func(c(FALSE, FALSE, FALSE)))
stopifnot(all_func(c(TRUE, TRUE, NA)))
stopifnot(!all_func(c(TRUE, FALSE, NA)))
stopifnot(all_func(c(NA, TRUE, NA)))

### 1.2

Now consider the function that does the opposite, returns `TRUE` if any of the elements are `TRUE`, and returns `FALSE` if all of the elements are `FALSE`.

Write a function `any_func` that does this. Hint: this function can be written extremely simply given a correct implementation of `all_func` or `all_iter` if you think carefully about what this is doing.

In [35]:
any_func = function(x) {
    !all_func(!x)
}

In [36]:
stopifnot(any_func(c(TRUE, TRUE, TRUE)))
stopifnot(any_func(c(TRUE, TRUE, FALSE)))
stopifnot(!any_func(c(FALSE, FALSE, FALSE)))
stopifnot(any_func(c(TRUE, TRUE, NA)))
stopifnot(any_func(c(TRUE, FALSE, NA)))
stopifnot(!any_func(c(FALSE, FALSE, NA)))
stopifnot(any_func(c(NA, TRUE, NA)))

Note that `any` and `all` are built-in `R` functions that do the same things as the above functions you wrote.

### 1.3

Write a function `exists_outlier` that takes a vector and checks if an outlier exists. We define an outlier to be a point that is more than 2.5x the standard deviation away from the mean.

In [50]:
exists_outlier = function(x) {
    any(abs(x - mean(x)) > 2.5*sd(x))
}

In [51]:
stopifnot(!exists_outlier(c(1, 2, 3, 4, 5)))
stopifnot(exists_outlier(c(rep(1, 20), 5)))

### 1.4

The following code creates a dataframe `dat`. Write a one-liner using `exists_outlier` that prints the variable names of `dat` that contain an outlier.

In [78]:
set.seed(123515)
dat = data.frame(X1=rnorm(30))
for (varn in 2:20) {
    dat[[paste0('X', varn)]] = rnorm(30)
}

In [79]:
names(dat)[map_lgl(dat, exists_outlier)] %>% print

[1] "X1"  "X2"  "X5"  "X14" "X17"


## Q2: Max and Argmax

### 2.1

Define the function `max_iter` that takes a vector and returns the maximum element using iteration/loops.

Define the function `max_func` that does the same thing using `reduce`. Note: don't use the `max` function, instead define your own function that goes into the reduce.

In [211]:
max_iter = function(x) {
    curr = x[1]
    
    for (i in 2:length(x)) {
        if (x[i] > curr) {
            curr = x[i]
        }
    }
    curr
}


In [212]:
reduce_max = function(curr, newval) {
    if (newval > curr) {
        return(newval)
    } else {
        return(curr)
    }
}

max_func = function(x) {
    reduce(x, reduce_max)
}

In [213]:
test = rnorm(100)
stopifnot(max_iter(test) == max(test))
stopifnot(max_func(test) == max(test))

### 2.2

Define the function `argmax_iter` that takes a vector and returns the index maximum element using iteration/loops. If there is a tie return the index of the first time the max appears. Hint: you may need to keep track of multiple things as you iterate.

(Difficult) Define the function `argmax_func` that does the same thing using reduce. Hint: how can you keep track of multiple things through the reduce steps?


In [214]:
argmax_iter = function(x) {
    curr_val = x[1]
    curr_arg = 1
    
    for (i in 2:length(x)) {
        if (x[i] > curr_val) {
            curr_val = x[i]
            curr_arg = i
        }
    }
    curr_arg
}

In [215]:
reduce_argmax = function(curr, newval) {
    if (newval > curr$val) {
        return(list(val=newval, arg=curr$i+1, i=curr$i+1))
    } else {
        return(list(val=curr$val, arg=curr$arg, i=curr$i+1))
    }
}

argmax_func = function(x) {
    res = reduce(x, reduce_argmax, .init=list(val=-Inf, arg=0, i=0))
    res$arg
}

In [216]:
test = rnorm(100)
stopifnot(argmax_iter(c(1, -3, 2, 6, 3)) == 4)
stopifnot(argmax_iter(test) == which.max(test))
stopifnot(argmax_func(c(1, -3, 2, 6, 3)) == 4)
stopifnot(argmax_func(test) == which.max(test))